In [ ]:
# Install the neccessary libraries 

!pip install bs4
!pip install selenium

In [2]:
# Impor the necessary libraries 

from bs4 import BeautifulSoup
from selenium import webdriver
import time

#You need to have driver corresponding to your Google Chrome browser or update browser
from selenium.webdriver.common.by import By

In [3]:
#this generates USNs from 1 to 330 
#If your branch is different then you can change the branch code such as BEC,BCV etc

usns=[]
for i in range(1,330):
    if i<=9:
        usn='01FE20BCS00'+str(i)
    elif i>=10 and i<=99:
        usn='01FE20BCS0'+str(i)
    else:
        usn='01FE20BCS'+str(i)
    usns.append(usn)
#For Diploma students
for i in range(1,33):
    if i<=9:
        usn='01FE21BCS40'+str(i)
    elif i>=10 and i<=99:
        usn='01FE21BCS4'+str(i)
    usns.append(usn)

Run the cell below and wait until Google Chrome opens.

In [4]:
url="https://eresults.kletech.ac.in/index.php"
# url="https://eresults.kletech.ac.in/kletecheven/index.php"

#This is the path name of chrome driver on my PC
# driver = webdriver.Chrome("C:/Users/vskon/Downloads/chromedriver_win32/chromedriver")
from selenium import webdriver
driver = webdriver.Chrome()

driver.get(url)
driver.maximize_window()

When the webpage opens in the chrome then change the captcha below and run the cell . KLE Tech. Result website has a glitch that its captcha doesn't change if we navigate back on the website.

In [5]:
#put the captcha as shown in website and run this cell
captcha="AD5Q9" #this was the current captcha when I was using

In [6]:
file=open('result7_sgpa_final.csv','a') #This is the file in which I am storing Name and SGPA

In [7]:
# Print the USN, name, SGPA, and CGPA heading
print('USN,Name,SGPA,CGPA', end='')
file.write('USN,Name,SGPA,CGPA')

# Print the subject names as headings
subjects = ['Constitution of India, Professional Ethics and Environmental Studies','Big Data and Analytics','Information Security','Senior Design Project','Social Network Analysis(Swayam)','Cyber Security and Privacy(Swayam)']
for subject in subjects:
    print(',' + subject, end='')
    file.write(',' + subject)
print()
file.write('\n')

# For each USN, scrape the details
for username in usns:
    driver.find_element(by=By.NAME,value="usn").send_keys(username)
    driver.find_element(by=By.NAME,value="osolCatchaTxt").send_keys(captcha)

    # driver.find_element_by_formaction("index.php?option=com_examresult&task=getResult").click()
    driver.find_element(By.XPATH, '//button[text()="Result"]').click()
    
    # Handle exceptions for USNs with no results or non-existent USNs
    try:
        # This //div..... may change depending on what you are trying to scrape
        name = driver.find_element(By.XPATH,'//div[contains(@class,"uk-card uk-card-body stu-data stu-data1")]')
        print(name.text) # optional
        
    except:
        print("Error in name")
    
    try:
        sgpa = driver.find_element(By.XPATH,'//div[contains(@class,"uk-card uk-card-default uk-card-body credits-sec3")]')
        print(sgpa.text[5:]) # Since the extracted data has SGPA as suffix but we need to store only numerical part
        file.write('\n' + username + ',' + name.text + ',' + sgpa.text[5:])
    except:
        print("Error in sgpa")

    try:
        cgpa = driver.find_element(By.XPATH,'//div[contains(@class,"uk-card uk-card-default uk-card-body credits-sec4")]/p')
        print(cgpa.text) # Since the extracted data has CGPA as suffix but we need to store only numerical part
        file.write(',' + cgpa.text)
    except:
        print("Error in cgpa")

    try:
        subject_xpath = '//table[@class="uk-table uk-table-striped res-table"]/tbody/tr/td[2]'
        grade_xpath = '//table[@class="uk-table uk-table-striped res-table"]/tbody/tr/td[5]'

        scraped_subjects = driver.find_elements(By.XPATH, subject_xpath)
        grades = driver.find_elements(By.XPATH, grade_xpath)

        # Create a dictionary of scraped subjects and their grades
        grades_dict = {}
        for subject, grade in zip(scraped_subjects, grades):
            grades_dict[subject.text] = grade.text

        # Print only the grades for each USN, or '-' if the subject is not present
        for subject in subjects:
            if subject in grades_dict:
                print(grades_dict[subject], end=',')
                file.write(',' + grades_dict[subject])
            else:
                print('-', end=',')
                file.write(',-')

    except:
        print("Error in subjects and grades")

    driver.back() # Navigating back by clicking back button

    driver.find_element(by=By.NAME,value="usn").clear() # Clearing the already filled detail
    driver.find_element(by=By.NAME,value="osolCatchaTxt").clear() # Clearing the CAPTCHA field as it will be entered again


USN,Name,SGPA,CGPA,Constitution of India, Professional Ethics and Environmental Studies,Big Data and Analytics,Information Security,Senior Design Project,Social Network Analysis(Swayam),Cyber Security and Privacy(Swayam)
ASHRITA ANAND GAONKAR
7.17
6.40
AP,D,E,A,C,C,SOUMYA KOTRESH KATAGIHALLI
8.67
8.31
AP,B,A,S,C,B,ADITYA DILIP JADHAV
Error in sgpa
Error in cgpa
-,-,-,-,-,-,PRERANA SATYANARAYANA MURTHY MALAGI
8.34
7.44
AP,D,A,A,B,A,AKSHATA PARAMESH BADNI
Error in sgpa
Error in cgpa
-,-,-,-,-,-,NEHA PATIL
8.84
8.11
AP,B,A,S,B,B,LOHIT VINAYAK GANGOLLI
Error in sgpa
Error in cgpa
-,-,-,-,-,-,KONKATHI RITHIN KUMAR
2.15
5.91
-,-,-,-,-,-,RANJITA PRASHANT HEGDE
8.50
7.80
AP,A,B,S,B,D,SRISHTI ANIL KADAM
8.75
7.77
AP,C,B,S,-,-,ULHAS S KALASA
8.34
7.68
AP,C,A,A,A,C,PRAVEEN VISHWA KULLI
8.84
7.96
AP,B,B,S,A,B,NIKHITA BALAJI NADAKARNI
Error in sgpa
Error in cgpa
-,-,-,-,-,-,SANDEEP SADANAND HALKATTI
8.34
8.04
AP,B,D,S,B,B,SAHANA G AIDNAL
9.17
7.45
AP,A,B,S,A,A,SRUSHTI PRAKASH NAYAK
8.50
7.42
AP,B,B

In [8]:
file.close() #closing the file